In [1]:
from __future__ import print_function

def data():

    import tensorflow as tf
    from tensorflow import keras
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
    import pandas as pd
    import pickle
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from keras import backend as K

    import warnings
    warnings.filterwarnings('ignore')

    path_origin, path_processed, models_path, cells = "../data/original_data/", "../data/processed_data/", "../data/processed_data/models/", [f'\Cell{x}\*' for x in range(1, 9)]

    df_desc_final_pickle_load = pd.read_pickle(path_processed + 'df_desc_final.pkl')
    df_desc_final_pickle = df_desc_final_pickle_load[df_desc_final_pickle_load.RUL >= 0 ]
    df_desc_final_pickle["RUL"] = df_desc_final_pickle["RUL"].astype(float)

    x, y, num_steps, percentage_and_rul = [], [], 10, []

    for i in range(1, 9):
        batery_df = df_desc_final_pickle[df_desc_final_pickle.index.get_level_values(0) == i]
        batery_df.index = pd.MultiIndex.from_tuples([(1, x) for x in range(1, batery_df.shape[0] + 1)])
        for j in range(1, len(batery_df) - num_steps):
            x.append(np.array(batery_df.loc[(1, slice(j,num_steps+j)), :].drop(columns=['RUL']).values))
            y.append(np.array([batery_df.loc[(1, slice(j,num_steps+j)), :].RUL.values[-1], i]))
            
    x, y = np.array(x), np.array(y)
    X_train,X_test,y_train_mod,y_test_mod = train_test_split(x, y,test_size=0.2, random_state=42, stratify=y[:,1])
    y_train, y_test = y_train_mod[:,0].reshape(-1,1), y_test_mod[:,0].reshape(-1,1)
    X_train, y_train = np.stack(X_train, axis = 0), np.stack(y_train, axis = 0)
    X_test, y_test = np.stack(X_test, axis = 0), np.stack(y_test, axis = 0)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    lista_resultados = []
    return X_train,  y_train,  lista_resultados 

def create_model(X_train, y_train, lista_resultados):

    import numpy as np
    import warnings
    warnings.filterwarnings('ignore')

    def customLoss(true,pred):
        diff = pred - true
        greater = K.greater(diff,0)
        greater = K.cast(greater, K.floatx()) 
        greater = greater + 1                
        return K.mean(K.square(diff))*greater
    
    nombre = models_path + 'tryals_bidirectional_lstm_' + str(len(lista_resultados)) + '.h5'
    checkpoint_rb = keras.callbacks.ModelCheckpoint(nombre, verbose=0, save_best_only=True)

    alphav1, alphav2 = 0, 0
    acti1v = {{ choice(['leaky' , 'prelu', 'tanh'])}}
    if acti1v == 'leaky':  
        alphav1 = {{uniform(0, 1)}}
        acti1 = keras.layers.LeakyReLU(alpha=alphav1)
    if acti1v == 'prelu': acti1 = tf.keras.layers.PReLU('zeros')
    if acti1v == 'relu': acti1 = 'relu'
    if acti1v == 'tanh': acti1 = 'tanh'

    acti2v = {{ choice(['leaky','prelu','tanh'])}}
    if acti2v == 'leaky':  
        alphav2 = {{uniform(0, 1)}}
        acti2 = keras.layers.LeakyReLU(alpha=alphav2)
    if acti2v == 'prelu': acti2 = tf.keras.layers.PReLU('zeros')
    if acti2v == 'relu': acti2 = 'relu'
    if acti2v == 'tanh': acti2 = 'tanh'

    kernel_initializer1, kernel_initializer2 = {{choice( ["glorot_uniform", tf.keras.initializers.Orthogonal()] )}}, {{choice( ["glorot_uniform", tf.keras.initializers.Orthogonal()] )}}
    recurrent_initializer1, recurrent_initializer2 = {{choice( ["glorot_uniform", tf.keras.initializers.Orthogonal()] )}}, {{choice( ["glorot_uniform", tf.keras.initializers.Orthogonal()] )}}
    recurrent_dropout1, recurrent_dropout2 = {{uniform(0, 1)}}, {{uniform(0, 1)}}
    neurons1, neurons2 = {{choice([x for x in range(256, 1024, 32)])}}, {{choice([x for x in range(256, 1024, 32)])}}
    np.random.seed(26), tf.random.set_seed(26);

    model = keras.Sequential()
    forward_layer = keras.layers.LSTM(neurons1, activation = acti1, kernel_initializer=kernel_initializer1, recurrent_initializer=recurrent_initializer1, 
                                        recurrent_dropout = recurrent_dropout1)
    backward_layer = keras.layers.LSTM(neurons2, activation = acti2, kernel_initializer=kernel_initializer2, recurrent_initializer=recurrent_initializer2, 
                                        recurrent_dropout = recurrent_dropout2, go_backwards = True)

    model.add(keras.layers.Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(X_train.shape[1], X_train.shape[2])  ))
    model.add(keras.layers.Dense(1, activation = 'relu'))

    lr = {{choice([0.005,0.001,0.0005, 0.0001])}}
    model.compile(loss = customLoss,  optimizer = keras.optimizers.Adam(learning_rate = lr), metrics = ['mse'])
    
    print('=======================================================================================================================================================================================')
    batch_size = {{choice([x for x in range(16, 128, 16)])}}
    history = model.fit(X_train, y_train,
              batch_size = batch_size,
              epochs=500, validation_split=0.2, verbose=0, callbacks = [checkpoint_rb])

    val_acc_per_epoch = history.history['val_loss']
    best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch))
    lista_resultados.append(history.history['val_loss'][best_epoch])
    print('El trial con menor error es el:', np.argmin(lista_resultados))
    params_dic = {'neurons1': neurons1, 'neurons2': neurons2, 'acti1v': acti1v, 'acti2v': acti2v, 'alphav1': alphav1, 'alphav2': alphav2, 'kernel_initializer1': kernel_initializer1,
                    'kernel_initializer2': kernel_initializer2, 'recurrent_initializer1': recurrent_initializer1, 'recurrent_initializer2': recurrent_initializer2,
                    'recurrent_dropout1': recurrent_dropout1, 'recurrent_dropout2': recurrent_dropout2, 'lr': lr, 'batch_size': batch_size}
    print(params_dic)
    
    return {'loss' : history.history['val_loss'][best_epoch], 'status' : STATUS_OK, 'model' : model, 'metrics' : np.sqrt(history.history['val_mse'][best_epoch])}

if __name__ == '__main__':
    from hyperas import optim
    from hyperas.distributions import choice, uniform
    from hyperopt import Trials, STATUS_OK, tpe
    X_train, y_train, lista_resultados = data()
    best_run, best_model= optim.minimize(model=create_model,
                                         data=data,
                                         algo=tpe.suggest,
                                         max_evals=75,
                                         trials=Trials(),
                                         notebook_name='bidirectional_lstm_hyperas')
    print(best_model, best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import pickle
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import warnings
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import warnings
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe

KeyboardInterrupt: 

In [4]:
best_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 360)               524520    
_________________________________________________________________
dropout_23 (Dropout)         (None, 360)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 361       
Total params: 524,881
Trainable params: 524,881
Non-trainable params: 0
_________________________________________________________________


In [3]:
best_run

{'Dropout': 0.02733288830118466,
 'LSTM': 5,
 'activ': 1,
 'alpha': 0.3799718452885261,
 'batch_size': 0,
 'constrain': 1,
 'kernel_initializer': 0,
 'max_value': 4.132771265163272}